In [9]:
import numpy as np
import scipy.io as scio
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import os
import skimage
import csv

In [3]:
hr_df  = pd.read_csv('HR_FullTimeSeries.csv')
hr_data = hr_df.drop(['Subject', 'FinalMS', '1350'], axis=1).to_numpy()

eda_df  = pd.read_csv('EDA_FullTimeSeries.csv')
eda_data = eda_df.drop(['Subject', 'FinalMS', '1350'], axis=1).to_numpy()


num_subjects = hr_data.shape[0]
length = hr_data.shape[1]

num_subjects = hr_data.shape[0]
length = hr_data.shape[1]

hr_data_normalized = np.divide(hr_data - np.mean(hr_data, axis=1, keepdims=True), np.std(hr_data, axis=1, keepdims=True))
eda_data_normalized = np.divide(eda_data - np.mean(eda_data, axis=1, keepdims=True), np.std(eda_data, axis=1, keepdims=True))

In [8]:
full_time_distances_hr = np.empty((num_subjects, num_subjects))
full_time_distances_eda = np.empty((num_subjects, num_subjects))

for i in range(num_subjects): # Set diagonals to zero
    for j in range(num_subjects):
        full_time_distances_hr[i,j] = 1/length * np.linalg.norm(hr_data_normalized[i,:] - hr_data_normalized[j,:], ord=2) ** 2
        full_time_distances_eda[i,j] = 1/length * np.linalg.norm(eda_data_normalized[i,:] - eda_data_normalized[j,:], ord=2) ** 2
    

In [11]:
np.savetxt("HR_Distances.csv", full_time_distances_hr, delimiter=",")
np.savetxt("EDA_Distances.csv", full_time_distances_hr, delimiter=",")

In [42]:
hr_blocks = np.squeeze(skimage.util.view_as_blocks(hr_data_normalized, block_shape = (num_subjects, 90)))
eda_blocks = np.squeeze(skimage.util.view_as_blocks(eda_data, block_shape = (num_subjects, 90)) )

block_distances_hr = np.empty((15,73,73))
block_distances_eda = np.empty((15,73,73))

out_hr = np.array([])
out_eda = np.array([])
for block in range(15):
    for i in range(num_subjects): # Set diagonals to zero
        for j in range(num_subjects):
    
            block_distances_hr[block,i,j] = 1/90 * np.linalg.norm(hr_blocks[block, i, :] - hr_blocks[block, j, :], ord = 2) ** 2
            block_distances_eda[block,i,j] = 1/90 * np.linalg.norm(eda_blocks[block, i, :] - eda_blocks[block, j, :], ord = 2) ** 2
    out_hr = np.hstack((out_hr,block_distances_hr[block,:,:])) if out_hr.size else block_distances_hr[block,:,:]
    out_eda = np.hstack((out_eda,block_distances_eda[block,:,:])) if out_eda.size else block_distances_eda[block,:,:]

In [28]:
block_distances_hr[0,:,:]

array([[0.        , 0.15182555, 0.20487959, ..., 0.5879311 , 0.29284925,
        0.52722303],
       [0.15182555, 0.        , 0.09535523, ..., 1.1418006 , 0.28315367,
        0.20502499],
       [0.20487959, 0.09535523, 0.        , ..., 1.10419533, 0.08487368,
        0.5200417 ],
       ...,
       [0.5879311 , 1.1418006 , 1.10419533, ..., 0.        , 0.83114768,
        2.0228976 ],
       [0.29284925, 0.28315367, 0.08487368, ..., 0.83114768, 0.        ,
        0.94743311],
       [0.52722303, 0.20502499, 0.5200417 , ..., 2.0228976 , 0.94743311,
        0.        ]])

In [43]:
out_eda.shape

(73, 1095)

In [44]:
np.savetxt("HR_Distances_Windows.csv", out_hr, delimiter=",")
np.savetxt("EDA_Distances_Windows.csv", out_eda, delimiter=",")